In [ ]:
import pandas as pd
import duckdb
import warnings
warnings.filterwarnings('ignore')

print("Starting optimized CAM-ICU and RASS analysis with medication cohort...")

# Setup DuckDB connection with memory limit
con = duckdb.connect()
con.execute("SET memory_limit='8GB'")

# Define item IDs
cam_itemids = '001C_1961_26610'
rass_itemids = '001C_1596_21550'

print("\nLoading medication cohort...")
cohort_df = pd.read_csv('icu_cohort_with_medication_features.csv')
print(f"Cohort loaded: {cohort_df.shape[0]:,} ICU stays")

# Convert datetime columns
cohort_df['icu_intime'] = cohort_df['icu_intime'].astype('datetime64[ms]')
cohort_df['icu_outtime'] = cohort_df['icu_outtime'].astype('datetime64[ms]')


In [ ]:

# Get list of stay IDs for filtering
stay_ids = cohort_df['icu_stay_id'].tolist()
print(f"Processing {len(stay_ids):,} ICU stays")

# Create a temporary table with cohort data
con.execute("CREATE TABLE cohort AS SELECT * FROM cohort_df")

# Process CAM-ICU data with simplified query
print("\nProcessing CAM-ICU data (this may take several minutes)...")

In [ ]:
chartevents = pd.read_csv('../KMIMIC/chartevents.csv')

In [ ]:
# Create new dataframe with the specified conditions
new_df = chartevents[(chartevents['item_id'] == '001C_1961_26610') & 
                     (chartevents['stay_id'].isin(stay_ids))]

In [ ]:
new_df['charttime'] = new_df['charttime'].astype('datetime64[ms]')

In [ ]:
new_df = new_df[["stay_id","charttime","value"]]

In [ ]:


# Filter by time bounds
print("Filtering CAM data by ICU stay times...")
con.execute("""
CREATE TABLE cam_filtered AS
SELECT 
    cd.*,
    date_trunc('hour', cd.charttime) as hour_time
FROM new_df cd
JOIN cohort c ON cd.stay_id = c.icu_stay_id
WHERE cd.charttime >= c.icu_intime 
    AND cd.charttime <= c.icu_outtime
""")


In [ ]:
cam_results = con.execute("""
SELECT 
    stay_id as icu_stay_id,
    COUNT(CASE WHEN value = 'positive' THEN 1 END) as cam_total_positive_count,
    COUNT(CASE WHEN value = 'Negative' THEN 1 END) as cam_total_negative_count,
    COUNT(CASE WHEN value = 'Unable to assess' THEN 1 END) as cam_total_Unable_to_assess_count
FROM cam_filtered
WHERE value IS NOT NULL
GROUP BY stay_id
""").fetchdf()

In [ ]:
# Create new dataframe with the specified conditions
rass_df = chartevents[(chartevents['item_id'] == '001C_1596_21550') & 
                     (chartevents['stay_id'].isin(stay_ids))]

In [ ]:
rass_df['charttime'] = rass_df['charttime'].astype('datetime64[ms]')

In [ ]:
rass_df = rass_df[["stay_id","charttime","value"]]

In [ ]:

# Process RASS data
print("\nProcessing RASS data...")

# Extract RASS data
rass_extract_query = f"""
CREATE TABLE rass_data AS
SELECT 
    c.stay_id,
    c.charttime::TIMESTAMP as charttime,
    c.value
FROM rass_df c
"""

print("Extracting RASS data from chartevents...")
con.execute(rass_extract_query)

In [ ]:

# Filter by time bounds and parse scores
print("Filtering and parsing RASS scores...")
rass_parsed_query = """
CREATE TABLE rass_parsed AS
SELECT 
    rd.stay_id,
    rd.charttime,
    CASE 
        WHEN rd.value ILIKE '%4%' OR rd.value ILIKE '%combative%' THEN 4
        WHEN rd.value ILIKE '%3%' OR rd.value ILIKE '%very agitated%' THEN 3
        WHEN rd.value ILIKE '%2%' OR rd.value ILIKE '%agitated%' THEN 2
        WHEN rd.value ILIKE '%1%' OR rd.value ILIKE '%restless%' THEN 1
        WHEN rd.value ILIKE '%0%' OR rd.value ILIKE '%alert and calm%' THEN 0
        WHEN rd.value ILIKE '%-1%' OR rd.value ILIKE '%drowsy%' THEN -1
        WHEN rd.value ILIKE '%-2%' OR rd.value ILIKE '%light sedation%' THEN -2
        WHEN rd.value ILIKE '%-3%' OR rd.value ILIKE '%moderate sedation%' THEN -3
        WHEN rd.value ILIKE '%-4%' OR rd.value ILIKE '%deep sedation%' THEN -4
        WHEN rd.value ILIKE '%-5%' OR rd.value ILIKE '%unarousable%' THEN -5
        ELSE TRY_CAST(REGEXP_EXTRACT(rd.value, '([+-]?\\d+)', 1) AS INTEGER)
    END as rass_score
FROM rass_data rd
JOIN cohort c ON rd.stay_id = c.icu_stay_id
WHERE rd.charttime >= c.icu_intime 
    AND rd.charttime <= c.icu_outtime
"""

con.execute(rass_parsed_query)

In [ ]:
# Aggregate RASS results
print("Aggregating RASS results by ICU stay...")
rass_results = con.execute("""
SELECT 
    stay_id as icu_stay_id,
    COUNT(rass_score) as rass_count,
    MIN(rass_score) as rass_min,
    MAX(rass_score) as rass_max,
    ROUND(AVG(rass_score)::NUMERIC, 2) as rass_mean,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY rass_score) as rass_median
FROM rass_parsed
WHERE rass_score IS NOT NULL 
    AND rass_score BETWEEN -5 AND 4
GROUP BY stay_id
""").fetchdf()


In [ ]:

# Combine results using pandas merge (more efficient for this operation)
print("\nCombining data with LEFT JOINs...")

# Merge CAM results
final_df = cohort_df.merge(cam_results, on='icu_stay_id', how='left')

# Merge RASS results
final_df = final_df.merge(rass_results, on='icu_stay_id', how='left')

# Fill missing counts with 0
final_df['cam_total_positive_count'] = final_df['cam_total_positive_count'].fillna(0).astype(int)
final_df['cam_total_negative_count'] = final_df['cam_total_negative_count'].fillna(0).astype(int)
final_df['rass_count'] = final_df['rass_count'].fillna(0).astype(int)

# Summary statistics
print(f"\nFinal dataset: {final_df.shape[0]:,} rows, {final_df.shape[1]} columns")
print(f"ICU stays with CAM assessments: {(final_df['cam_total_positive_count'] + final_df['cam_total_negative_count'] > 0).sum():,}")
print(f"ICU stays with RASS assessments: {(final_df['rass_count'] > 0).sum():,}")

# Distribution of CAM assessments
cam_positive_stays = (final_df['cam_total_positive_count'] > 0).sum()
cam_negative_stays = (final_df['cam_total_negative_count'] > 0).sum()
print(f"\nCAM-ICU distribution:")
print(f"  Stays with ≥1 positive assessment: {cam_positive_stays:,}")
print(f"  Stays with ≥1 negative assessment: {cam_negative_stays:,}")

# RASS statistics
rass_stays = final_df[final_df['rass_count'] > 0]
if len(rass_stays) > 0:
    print(f"\nRASS statistics:")
    print(f"  Mean RASS count per stay: {rass_stays['rass_count'].mean():.1f}")
    print(f"  Mean RASS score: {rass_stays['rass_mean'].mean():.2f}")
    print(f"  Median RASS score: {rass_stays['rass_median'].median():.2f}")

# Save the final dataset
output_path = 'icu_cohort_medication_cam_rass.csv'
final_df.to_csv(output_path, index=False)
print(f"\nFinal dataset saved to: {output_path}")

# Show sample of results
print("\nSample of final dataset (first 5 rows with new columns):")
sample_cols = ['icu_stay_id', 'cam_total_positive_count', 'cam_total_negative_count', 
               'rass_count', 'rass_min', 'rass_max', 'rass_mean', 'rass_median']
print(final_df[sample_cols].head())

# Close connection
con.close()
print("\nAnalysis complete!")